## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,75.09,70,6,7.58,clear sky
1,1,Tabou,CI,4.42,-7.35,79.30,83,66,6.33,broken clouds
2,2,Saint George,US,37.10,-113.58,57.20,24,1,1.63,clear sky
3,3,Dolores,AR,-36.31,-57.68,70.47,44,0,2.37,clear sky
4,4,Balao,EC,-2.92,-79.82,80.60,69,75,8.05,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,75.09,70,6,7.58,clear sky
1,1,Tabou,CI,4.42,-7.35,79.30,83,66,6.33,broken clouds
3,3,Dolores,AR,-36.31,-57.68,70.47,44,0,2.37,clear sky
4,4,Balao,EC,-2.92,-79.82,80.60,69,75,8.05,broken clouds
6,6,Kapaa,US,22.08,-159.32,75.20,83,1,5.82,clear sky
8,8,Puerto Ayora,EC,-0.74,-90.35,77.00,69,75,9.17,broken clouds
12,12,Solwezi,ZM,-12.17,26.39,71.98,29,9,3.89,clear sky
14,14,Vaini,TO,-21.20,-175.20,73.40,88,75,9.17,broken clouds
17,17,Kahului,US,20.89,-156.47,80.60,61,1,4.18,clear sky
20,20,Belmonte,BR,-15.86,-38.88,82.40,69,75,18.34,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                256
City                   256
Country                252
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,75.09,70,6,7.58,clear sky
1,1,Tabou,CI,4.42,-7.35,79.30,83,66,6.33,broken clouds
3,3,Dolores,AR,-36.31,-57.68,70.47,44,0,2.37,clear sky
4,4,Balao,EC,-2.92,-79.82,80.60,69,75,8.05,broken clouds
6,6,Kapaa,US,22.08,-159.32,75.20,83,1,5.82,clear sky
...,...,...,...,...,...,...,...,...,...,...
679,679,San Jose,US,37.34,-121.89,71.01,48,20,2.53,haze
682,682,Manaus,BR,-3.10,-60.02,89.60,62,40,5.82,scattered clouds
683,683,Banda Aceh,ID,5.56,95.32,80.60,75,98,6.96,light rain
686,686,Myitkyina,MM,25.38,97.40,70.81,96,73,2.51,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.09,clear sky,-23.12,-134.97,
1,Tabou,CI,79.30,broken clouds,4.42,-7.35,
3,Dolores,AR,70.47,clear sky,-36.31,-57.68,
4,Balao,EC,80.60,broken clouds,-2.92,-79.82,
6,Kapaa,US,75.20,clear sky,22.08,-159.32,
8,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,
12,Solwezi,ZM,71.98,clear sky,-12.17,26.39,
14,Vaini,TO,73.40,broken clouds,-21.20,-175.20,
17,Kahului,US,80.60,clear sky,20.89,-156.47,
20,Belmonte,BR,82.40,broken clouds,-15.86,-38.88,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.09,clear sky,-23.12,-134.97,Pension Maro'i
1,Tabou,CI,79.30,broken clouds,4.42,-7.35,hotêl doufoulougou
3,Dolores,AR,70.47,clear sky,-36.31,-57.68,Apart Hotel - Arianos
4,Balao,EC,80.60,broken clouds,-2.92,-79.82,Gilson Stalin Lucas S
6,Kapaa,US,75.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
679,San Jose,US,71.01,haze,37.34,-121.89,The Westin San Jose
682,Manaus,BR,89.60,scattered clouds,-3.10,-60.02,Hotel do Largo
683,Banda Aceh,ID,80.60,light rain,5.56,95.32,OYO 884 Rumoh PMI Hotel
686,Myitkyina,MM,70.81,light rain,25.38,97.40,Shwe Phyu Hotel


In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.09,clear sky,-23.12,-134.97,Pension Maro'i
1,Tabou,CI,79.30,broken clouds,4.42,-7.35,hotêl doufoulougou
3,Dolores,AR,70.47,clear sky,-36.31,-57.68,Apart Hotel - Arianos
4,Balao,EC,80.60,broken clouds,-2.92,-79.82,Gilson Stalin Lucas S
6,Kapaa,US,75.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
8,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
12,Solwezi,ZM,71.98,clear sky,-12.17,26.39,Country Lodge Solwezi
14,Vaini,TO,73.40,broken clouds,-21.20,-175.20,Keleti Beach Resort
17,Kahului,US,80.60,clear sky,20.89,-156.47,Maui Seaside Hotel
20,Belmonte,BR,82.40,broken clouds,-15.86,-38.88,Pousada Monte Carmelo O Pão


In [11]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))